In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam

from keras import backend as K
from keras import initializers

In [ ]:
K.set_image_dim_ordering('th')

# Deterministic output.
# Tired of seeing the same results every time? Remove the line below.
np.random.seed(1000)

# The results are a little better when the dimensionality of the random vector is only 10.
# The dimensionality has been left at 100 for consistency with other GAN implementations.
randomDim = 100

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(60000, 784)

# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

generator = Sequential()
generator.add(
    Dense(
        256,
        input_dim=randomDim,
        kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

discriminator = Sequential()
discriminator.add(
    Dense(
        1024,
        input_dim=784,
        kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

# Combined network
discriminator.trainable = False
ganInput = Input(shape=(randomDim, ))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

dLosses = []
gLosses = []


# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/loss_epoch_%06d.png' % epoch)


# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/generated_image_epoch_%06d.png' % epoch)


# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator.save('models/generator_epoch_%06d.h5' % epoch)
    discriminator.save('models/discriminator_epoch_%06d.h5' % epoch)


def createNoise(generatedImages, thresholding, depth):
    if depth > 20:
        return generatedImages
    noise_samples = discriminator.predict(generatedImages)
    flag = False
    for index, ns in enumerate(noise_samples):
        if ns > thresholding:
            flag = True
            generatedImages[index] = generator.predict(
                np.random.normal(0, 1, size=[1, randomDim]))
    if flag == True:
        generatedImages = createNoise(generatedImages, thresholding, depth + 1)
    return generatedImages


def set_step(epoch):
    dloss_npy = np.load('Loss/dynamic_D_loss.npy')
    gloss_npy = np.load('Loss/dynamic_G_loss.npy')
    loss_all = dloss_npy[-1] + gloss_npy[-1]
    d_step = dloss_npy[-1] / loss_all
    g_step = gloss_npy[-1] / loss_all
    return int(d_step * 20), int(g_step * 20)


def train(epochs=1, batchSize=128, thresholding=0.95):
    batchCount = X_train.shape[0] / batchSize
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)
    if not os.path.exists('images'):
        os.makedirs('images')
    if not os.path.exists('models'):
        os.makedirs('models')
    if not os.path.exists('Loss'):
        os.makedirs('Loss')

    for e in range(1, epochs + 1):
        print('-' * 15, 'Epoch %d' % e, '-' * 15)
        #tq = tqdm(range(int(batchCount)))
        #for _ in tqdm(range(0,int(batchCount),10)):
        dloss_l = []
        gloss_l = []

        if e <= 20:
            d_step, g_step = 10, 10
        else:
            d_step, g_step = set_step(e)
        for _ in tqdm(range(0, int(batchCount), 20)):
            # Get a random set of input noise and images
            for i in range(d_step):
                noise = np.random.normal(0, 1, size=[batchSize, randomDim])
                imageBatch = X_train[np.random.randint(
                    0, X_train.shape[0], size=batchSize)]
                #import pdb;pdb.set_trace()
                # Generate fake MNIST images
                generatedImages = generator.predict(noise)

                generatedImages = createNoise(generatedImages, thresholding, 1)
                #import pdb;pdb.set_trace()

                #print(np.shape(imageBatch), np.shape(generatedImages))
                X = np.concatenate([imageBatch, generatedImages])
                #import pdb;pdb.set_trace()
                # Labels for generated and real data
                yDis = np.zeros(2 * batchSize)
                # One-sided label smoothing
                yDis[:batchSize] = 1

                # Train discriminator
                discriminator.trainable = True
                dloss = discriminator.train_on_batch(X, yDis)
                dloss_l.append(dloss)
            # Train generator
            discriminator.trainable = False
            for j in range(g_step):
                noise = np.random.normal(0, 1, size=[batchSize, randomDim])
                yGen = np.ones(batchSize)
                gloss = gan.train_on_batch(noise, yGen)
                gloss_l.append(gloss)
        # Store loss of most recent batch from this epoch
        dLosses.append(np.mean(dloss_l))
        gLosses.append(np.mean(gloss_l))
        print("D_loss:", dLosses[e - 1])
        print("G_loss:", gLosses[e - 1])
        np.save("Loss/dynamic_D_loss.npy", dLosses)
        np.save("Loss/dynamic_G_loss.npy", gLosses)
        if e == 1 or e % 10 == 0:
            plotGeneratedImages(e)
            saveModels(e)

        if e % 50 == 0:
            plotLoss(e)

    # Plot losses from every epoch
    plotLoss(e)


if __name__ == '__main__':
    train(20000, 128)